In [ ]:

import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
filtered_df = pd.read_csv("../midsave/Africa_annotation_data_new.csv"


)


filtered_df.info()




In [ ]:
def quality_score(value, high_quality_value):
    if value == high_quality_value:
        return 2
    elif value == 'N/A':
        return None
    else:
        return 0

# Applying the quality scoring
qa_df = filtered_df
qa_df['Road_Score'] = qa_df['Road'].apply(lambda x: quality_score(x, 'No'))
qa_df['Building_Score'] = qa_df['Buidling'].apply(lambda x: quality_score(x, 'No'))
qa_df['Tree_Growth_Score'] = qa_df['Tree Growth Visible Over Time'].apply(lambda x: quality_score(x, 'Yes'))
qa_df['Geometry_Score'] = qa_df['Geometry Capturing Planting Area'].apply(lambda x: quality_score(x, 'Yes'))
qa_df['Forest_Score'] = qa_df['Forest at Planting'].apply(lambda x: quality_score(x, 'No'))
qa_df['location_Score'] = qa_df['Change in buffer region'].apply(lambda x: quality_score(x, 'No'))

# Calculating the Quality Index
qa_df['Quality_Index'] = qa_df[
    ['Road_Score', 'Building_Score', 'Tree_Growth_Score',
     'Geometry_Score', 'Forest_Score','location_Score']
].mean(axis=1)

#

qa_df.head()

In [ ]:
qa_df.info()

In [ ]:
qa_df.info()

In [ ]:
#Reforestation Dataset
file_path = "../midsave/submission_consolidated_reforestation_projects.parquet"


automatic_qframework = gpd.read_parquet(file_path)

automatic_qframework.info()

In [ ]:
automatic_qframework['site_id_created']

In [ ]:
merged_df = pd.merge(qa_df, automatic_qframework, on="site_id_created", how="inner", suffixes=('', '_duplicate'))


duplicates = [col for col in merged_df.columns if col.endswith('_duplicate') or col.endswith('_duplicate2') or col.endswith('_duplicate3')]
merged_df.drop(columns=duplicates, inplace=True)
merged_df = merged_df.head(250)
merged_df['Road_presence'] = merged_df['Road_presence'].astype(str).replace({'nan': 'No'})
merged_df.info()

In [ ]:
merged_df.head()

In [ ]:
print(merged_df['Road'].unique())
print(merged_df['Road_presence'].unique())

In [ ]:

merged_df['Road_presence'] = merged_df['Road_presence'].replace({'1.0': 'No', '0.0': 'Yes'})


In [ ]:

cm = confusion_matrix(merged_df['Road'], merged_df['Road_presence'], labels=['Yes', 'No'])
f1 = f1_score(merged_df['Road'], merged_df['Road_presence'], labels=['Yes', 'No'], average='weighted')
print(" Forest at Planting presence F1 Score:", f1)
accuracy = accuracy_score(merged_df['Road'], merged_df['Road_presence'])
print("Forest at Planting:", accuracy)
# Compute confusion matrix
cm = confusion_matrix(merged_df['Road'], merged_df['Road_presence'], labels=['Yes', 'No'])

colors = ["#deebf7", "#9ecae1", "#6baed6", "#3182bd", "#08519c"]
cmap = LinearSegmentedColormap.from_list("blue_shades", colors, N=256)

# Plot
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, linewidths=0.5,
                      xticklabels=['Yes', 'No'], yticklabels=['Yes', 'No'],
                      annot_kws={"size": 18})

# Get colorbar and set font size
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=18)

# Labels
plt.xlabel('Predicted (Automated)', fontsize=20)
plt.ylabel('Actual (Manual)', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.show()

In [ ]:

merged_df['Built_area_presence'] = merged_df['Built_area_presence'].replace({1.: 'No', 0.: 'Yes'})

In [ ]:
print(merged_df['Buidling'].unique())
print(merged_df['Built_area_presence'].unique())

In [ ]:

merged_df = merged_df.dropna(subset=['Buidling', 'Built_area_presence'])


cm = confusion_matrix(merged_df['Buidling'], merged_df['Built_area_presence'], labels=['Yes', 'No'])
f1 = f1_score(merged_df['Buidling'], merged_df['Built_area_presence'], labels=['Yes', 'No'], average='weighted')
print(" Buidling presence F1 Score:", f1)
accuracy = accuracy_score(merged_df['Buidling'], merged_df['Built_area_presence'])
print(" Buidling presence Accuracy:", accuracy)
colors = ["#deebf7", "#9ecae1", "#6baed6", "#3182bd", "#08519c"]
cmap = LinearSegmentedColormap.from_list("blue_shades", colors, N=256)


# Plot
plt.figure(figsize=(8, 6))
heatmap=sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, linewidths=0.5,
            xticklabels=['Yes', 'No'], yticklabels=['Yes', 'No'], annot_kws={"size": 18})
# Get colorbar and set font size
cbar = heatmap.collections[0].colorbar
cbar.ax.tick_params(labelsize=18)
# Labels
plt.xlabel('Predicted (Automated)', fontsize=20)
plt.ylabel('Actual (Manual)', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.show()